### Format the Google BigQuery query

It's strongly recommendeded that you first test your query in Google's BigQuery Sandbox (online). It returns results ~100 times faster, will yell at you pre-runtime if you introduce syntax errors, and it also shows a preview of expected data usage for the given query.

TODO: extend query to include Oct/Nov/Dec 2021, not just 2022+ [completed in data dump version]

In [ ]:
QUERY = """
SELECT
  q.id,
  q.title,
  q.body,
  q.accepted_answer_id,
  q.view_count,
  q.tags,
  q.answer_count,
  q.score AS question_score,
  a.score AS answer_score,
  a.body AS stackoverflow_answer
FROM
  bigquery-public-data.stackoverflow.posts_questions q
LEFT JOIN
  bigquery-public-data.stackoverflow.posts_answers a
ON
  q.accepted_answer_id = a.id
WHERE
  q.answer_count > 0
  AND q.accepted_answer_id > 0
  AND EXTRACT(YEAR FROM q.creation_date) >= 2022
  AND a.score >= 0
"""

### Fetch dataset using Google BigQuery

Authentication info: https://googleapis.dev/python/google-api-core/latest/auth.html (the query will fail if you don't set up authentication first).

To use server-side sampling with BigQuery (culls data server-side, reducing download size), use `TABLESAMPLE SYSTEM` e.g.:

`bigquery-public-data.stackoverflow.posts_questions q TABLESAMPLE SYSTEM (10 PERCENT)`

`bigquery-public-data.stackoverflow.posts_answers a TABLESAMPLE SYSTEM (50 PERCENT)`

(be aware that using less than 100% for both of questions and answers can result in no matches between the two if unlucky with rng, causing the query to return no results)

In [ ]:
%pip install google-cloud-bigquery
%pip install db-dtypes

import db_dtypes
import json
import os
import pandas as pd
from pathlib import Path
from google.cloud import bigquery as bq

# Get project name from local secrets.json file
def load_project_ID(secrets_file="secrets.json"):
    with open(secrets_file) as f:
        secrets = json.load(f)
    return secrets["BQ_PROJECT_ID"]

# Load a saved copy of the dataset from local disk (if it exists), otherwise run query and save it
PICKLED_RESULT = "dataset_pickled.pkl"      # The file name of the saved dataset (saved on / loaded from local disk)
cwd = Path().absolute()                     # Current working directory (note: possibly different from execution directory)

# See if we have a copy of the dataset on local disk; if we do load that
try:
    dataset_path = os.path.join(cwd, PICKLED_RESULT)
    results = pd.read_pickle(dataset_path)
    print("Saved copy of dataset loaded from local disk.")

# If not, run the query (tested runtime was up to 1 hour for full query!)
except FileNotFoundError:
    print("Saved copy of dataset (" + PICKLED_RESULT + ") not found - running query...")

    # prepare query
    client = bq.Client(project=load_project_ID())

    # run the query and save result to a dataframe
    results = client.query(QUERY.format(offset=0)).result().to_dataframe()
    print("Dataset has been downloaded...")

    # pickle the dataframe for persistent copy on local storage
    results.to_pickle(PICKLED_RESULT)
    print("Dataset has been pickled and saved to local storage. File name: " + PICKLED_RESULT)

# Yell if there's an *unexpected* error
except:
    raise Exception("Error while loading dataset.")

# Dump some extra info
print("Number of questions:", len(results))
results.head()